In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import os
import selenium
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [116]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dronai/' 
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [117]:
bs=BeautifulSoup(source)

In [118]:
sk=bs.find('div', {'class':'GRID_ITEM'}).find('a')
print(sk['href'])
img=sk.find('img')
print(img['title'])

/dronai/dronas-dji-mini-2-se-fly-more-combo--23881156.html
Dronas DJI Mini 2 SE Fly More Combo


In [119]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://kauno.diena.lt/' 
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [120]:

bs = BeautifulSoup(source, 'html.parser')

In [121]:
straipsniai = bs.find_all('div', {'class':'category-block-row grid-12 alpha omega unscore'})
print(len(straipsniai))

41


In [122]:
pavadinimai, nuorodos = [], []
for s in straipsniai:
    i = s.find('div', {'class':'articles-list-title-wrapper'})
    if i is not None:
        t = i.find('a', {'class':'articles-list-title'})
        print('::REM::')
        print(t)
        print('HREF?')
        print(t['href'])
        print(t.text)
        pavadinimai.append(t.text)
        nuorodos.append(t['href'])

::REM::
<a class="articles-list-title" href="naujienos/lietuva/politika/lietuva-renka-nauja-valdzia-apylinkes-uzsidare-skaiciuojami-balsai-1195621" style="font-size: 16px;line-height: 1.2;margin-bottom: 10px;display: inline;">2024-ųjų Seimo rinkimai: laimėjo socdemai, antri – TS-LKD, treti – „Nemuno Aušra“ </a>
HREF?
naujienos/lietuva/politika/lietuva-renka-nauja-valdzia-apylinkes-uzsidare-skaiciuojami-balsai-1195621
2024-ųjų Seimo rinkimai: laimėjo socdemai, antri – TS-LKD, treti – „Nemuno Aušra“ 
::REM::
<a class="articles-list-title" href="naujienos/lietuva/politika/laukia-kova-vienmandatese-aiskeja-i-antraji-tura-pateke-kandidatai-1195683" style="font-size: 16px;line-height: 1.2;margin-bottom: 10px;display: inline;">Į antrąjį Seimo rinkimų turą patekę kandidatai <span style="color:red;">(apžvalga)</span></a>
HREF?
naujienos/lietuva/politika/laukia-kova-vienmandatese-aiskeja-i-antraji-tura-pateke-kandidatai-1195683
Į antrąjį Seimo rinkimų turą patekę kandidatai (apžvalga)
::REM::
<a

In [123]:
zodziai=0
zodziukiekiai=[]
for p in pavadinimai:
    zodziai=zodziai + len(p.split())
    zodziukiekiai.append(len(p.split()))
    
print('vidutinis žodžių kiekis straipsnio pavadinime', round(zodziai/len(pavadinimai),2))
print(zodziukiekiai)

vidutinis žodžių kiekis straipsnio pavadinime 8.52
[12, 8, 7, 10, 10, 6, 11, 9, 10, 8, 14, 10, 7, 9, 6, 12, 13, 11, 6, 9, 5, 5, 5, 5, 9, 14, 3, 7, 6]


In [124]:
SDB = sqlite3.connect('kaunodiena.db') # jei neegsiztuoja - bus sukurfta nauja sqlite DB
Cs = SDB.cursor()

sql = '''create table if not exists T1
(
pavadinimas text not null,
nuoroda text not null,
zodziukiekis integer not null
)
'''
Cs.execute(sql)

sql_template = '''insert into T1 values (?,?,?)'''
for a, b, c in zip(pavadinimai, nuorodos, zodziukiekiai):
    Cs.execute(sql_template, (a, b, c))

SDB.commit() # commit() būtinas, jei norite įrašyti duoenis į DB

SDB.close()

In [125]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr=1' 
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [126]:
bs = BeautifulSoup(source, 'html.parser')

In [139]:
automobiliai = bs.find_all('a', {'class':'announcement-item'})
print(len(automobiliai))

20


In [141]:
pavadinimai, metai, kebulas, kaina, kuras, pavaros, galingumas, rida, regsalis, nuorodos = [], [], [], [], [], [], [], [], [], []
for auto in automobiliai:
    
    pavadinimai.append(auto.find('div', {'class':'announcement-title'}).get_text().strip())
    metai=auto.find('div', {'class':'announcement-parameters'}).find_all('span')[0].get_text()
    kebulas.append(auto.find('div', {'class':'announcement-parameters'}).find_all('span')[1].get_text())
    kaina.append(int(auto.find('div', {'class':'announcement-pricing-info'}).find('strong').get_text().strip().replace(' ','').replace('€', '')))
    a=auto.find('div', {'class':'announcement-parameters-block'}).find_all('span')
    kuras.append(a[0].get_text().strip())
    pavaros.append(a[1].get_text())
    galingumas.append(a[2].get_text().strip())
    rida.append(int(a[3].get_text().replace(' km', '').replace(' ','')))
    nuorodos.append(auto['href'])
    
    
   
    
print(pavadinimai)
print(metai)
print(kebulas)
print(kaina)
print(kuras)
print(pavaros)
print(galingumas)
print(rida)
print(nuorodos)


['BMW 428 Gran Coupe', 'BMW 530', 'BMW 530', 'BMW 530', 'Fiat 500X', 'BMW 320 Gran Turismo', 'Mazda CX-5', 'Mercedes-Benz CLK320', 'BMW 520', 'Volkswagen Amarok', 'Land Rover Discovery Sport', 'Toyota Prius', 'Mercedes-Benz C220', 'Peugeot 508', 'BMW 318', 'Volvo V70', 'BMW 530', 'BMW 525', 'BMW 325', 'BMW X5']
2010-07
['Kupė (Coupe)', 'Sedanas', 'Sedanas', 'Sedanas', 'Visureigis / Krosoveris', 'Hečbekas', 'Visureigis / Krosoveris', 'Kupė (Coupe)', 'Universalas', 'Pikapas', 'Visureigis / Krosoveris', 'Hečbekas', 'Universalas', 'Universalas', 'Universalas', 'Universalas', 'Sedanas', 'Universalas', 'Kupė (Coupe)', 'Visureigis / Krosoveris']
[18000, 26900, 48500, 4500, 10500, 14500, 19999, 1500, 4000, 13500, 39111, 3101, 3399, 3999, 4850, 5470, 5500, 9500, 9700, 9850]
['Benzinas', 'Benzinas', 'Benzinas', 'Benzinas', 'Benzinas / dujos', 'Dyzelinas', 'Benzinas', 'Benzinas / dujos', 'Dyzelinas', 'Dyzelinas', 'Dyzelinas / elektra', 'Benzinas / elektra', 'Dyzelinas', 'Dyzelinas', 'Dyzelinas', 

In [159]:

driver1 = webdriver.Chrome(options=opcijos)
url1 = 'https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html' 
driver1.get(url1)

source1 = driver1.page_source
driver1.close()
bs1 = BeautifulSoup(source1, 'html.parser')

In [168]:
print(bs1.find_all('li', {'class':'crumb'})[2].get_text().strip())
print(bs1.find_all('li', {'class':'crumb'})[3].get_text().strip())
xz=(bs1.find_all('div', {'class':'parameter-row'}))
print(list(xz))



BMW
428 Gran Coupe
[<div class="parameter-row">
<div class="parameter-value-inbank">
<div class="apl-price-calculator js-leasing-price-calculator apl-segment-1" data-amount="18000" data-announcement-id="26837939" data-category-id="2" data-platform="desktop">
<div class="price-col js-apl-period">
<select class="loan-term" title="Terminas">
<option value="6">
                    3158 € / 6 mėn.
                </option>
<option value="9">
                    2126 € / 9 mėn.
                </option>
<option value="12">
                    1610 € / 12 mėn.
                </option>
<option value="15">
                    1300 € / 15 mėn.
                </option>
<option value="18">
                    1094 € / 18 mėn.
                </option>
<option value="21">
                    946 € / 21 mėn.
                </option>
<option value="24">
                    836 € / 24 mėn.
                </option>
<option value="27">
                    750 € / 27 mėn.
                </option>
<o